In [1]:
import numpy as np
import cv2

## Zadanie 1 (popraw parametry)

In [15]:
def median(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    BUF = np.zeros((240,320,60),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.median(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

def average(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    BUF = np.zeros((240,320,60),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.average(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [16]:
median("highway", 5, 36, 3, 3, 11)
average("highway", 5, 36, 3, 3, 11)

P =  0.3227567358581717
R =  0.8241729856534716
F1 =  0.4638599517351471
P =  0.24330583118454424
R =  0.7608577949899757
F1 =  0.3687071178399809


## Zadanie 2

In [2]:
def median_nonbuf(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    mask = np.zeros((240,320),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        mask = np.where(mask < I, mask + 1, np.where(mask > I, mask - 1, mask))

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_diff = cv2.absdiff(I, mask)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [3]:
median_nonbuf("highway", 1, 36, 5, 3, 11)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/mateusz/Studia/MachineLearning/mlvenv/lib/python3.12/site-packages/cv2/qt/plugins"


P =  0.3180992885379803
R =  0.42842664803939184
F1 =  0.36511045431820044
